<a href="https://colab.research.google.com/github/PierrePinson/ML-challenge-Owkin/blob/master/Main_Notebook_Owkin_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#give access to GDrive
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


### Installs and imports

In [0]:
#Clone git, might take a minute cause data is included
#uncomment the lines below for first use
#%cd /content/gdrive/My Drive/
#! git clone https://github.com/PierrePinson/ML-challenge-Owkin.git

%cd /content/gdrive/My Drive/ML-challenge-Owkin/
!git pull #update custom modules

!pip install lifelines
!pip install random-survival-forest
!pip install pysurvival
!pip install lasagne
!pip install --upgrade https://github.com/Theano/Theano/archive/master.zip
!pip install --upgrade https://github.com/Lasagne/Lasagne/archive/master.zip
#!git clone https://github.com/jaredleekatzman/DeepSurv.git

#print time elapsed at each cell execution
!pip install ipython-autotime
%load_ext autotime 

%cd DeepSurv
!pip install .
%cd /content/gdrive/My Drive/ML-challenge-Owkin/

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import extract_features as ef #module to extract features from dataset
import lifelines #python library for survival analysis
from importlib import reload #reload extract_features when changes are made in git
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
import sys
sys.path.append('/content/gdrive/My Drive/ML-challenge-Owkin/DeepSurv/deepsurv')
import deep_surv
from deepsurv_logger import DeepSurvLogger, TensorboardLogger
import utils
import viz
import lasagne
reload(ef)

<module 'extract_features' from '/content/gdrive/My Drive/ML-challenge-Owkin/extract_features.py'>

time: 708 ms


#Get data


In [214]:
X = ef.extract_features('x_train/').drop("PatientID", axis=1)
Y = pd.read_csv('output_VSVxRFU.csv').drop("PatientID", axis=1)
#Split data in train and validation sets: 
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.15)
x_test = ef.extract_features('x_test/')
PatientID = x_test.PatientID.to_frame()
x_test = x_test.drop("PatientID", axis=1)

/content/gdrive/My Drive/ML-challenge-Owkin/extract_features.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


time: 1.93 s


#Compare Models

##Methode 1: Classic Random Forest

In [67]:
#train
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

rf_predictions = clf.predict(X_val)
concordance_index(Y_val.SurvivalTime, rf_predictions[:,0], Y_val.Event)

0.5971849865951743

time: 955 ms


##Method 2: DeepSurv

In [171]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel
model = NonLinearCoxPHModel([ {'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 128}], auto_scaler=True)
model.times = np.arange(3500)
X =  x_train
E =  y_train.Event
T = y_train.SurvivalTime
model.fit(X, T, E, init_method = 'glorot_uniform', optimizer ='adam',lr = 1e-4, num_epochs = 200, dropout = 0.2, verbose=True, batch_normalization=True)

prct = 1/2
hazard_pysurv = model.predict_survival (x_val)
prediction_pysurv = np.apply_along_axis(lambda x : model.times[np.argmax(x<prct)], 1, hazard_pysurv)
prediction_pysurv[prediction_pysurv==model.times[0]] = 700
concordance_index(y_val.SurvivalTime, prediction_pysurv.astype(int), y_val.Event)

0.6324655436447167

time: 1.71 s


##Method 3: Random survival forest

In [207]:
from pysurvival.models.survival_forest import RandomSurvivalForestModel

n_tree = 200
model = RandomSurvivalForestModel(num_trees = n_tree)
X =  x_train
E =  y_train.Event
T = y_train.SurvivalTime
model.fit(X, T, E)

prct = 1/2
hazard_rsf = model.predict_survival(x_val)
prediction_rsf = np.apply_along_axis(lambda x : model.times[np.argmax(x<prct)], 1, hazard_rsf)
prediction_rsf[prediction_rsf==model.times[0]] = 700
print("C_index :" +str(concordance_index(y_val.SurvivalTime, prediction_rsf.astype(int), y_val.Event)))


C_index :0.511938202247191
time: 2.26 s


## Method 4: Lifelines (Cox model)

In [210]:
#reduce number of feaure to avoid collinearities
nc_x_train = ef.calculate_vif_(x_train, thresh=20) 
non_collinear_features = x_train.keys()
##Other method : use PCA to reduce features
#from sklearn.decomposition import PCA
#n_components = 17 #empirical optimal value 

#pca = PCA(n_components=n_components)
#pca.fit(x_train.T)
#nc_x_train = pd.DataFrame(data = pca.components_.T)

Remaining variables:
Index(['original_shape_Compactness2', 'original_shape_Maximum3DDiameter',
       'original_shape_SurfaceVolumeRatio', 'original_firstorder_Energy',
       'original_firstorder_Kurtosis', 'original_firstorder_Median',
       'original_firstorder_Skewness', 'original_glcm_ClusterProminence',
       'original_glcm_ClusterShade', 'original_glcm_Contrast',
       'original_glcm_JointEnergy', 'original_glrlm_GrayLevelNonUniformity',
       'original_glrlm_RunLengthNonUniformity',
       'original_glrlm_ShortRunLowGrayLevelEmphasis',
       'original_glrlm_LongRunLowGrayLevelEmphasis',
       'original_glrlm_LongRunHighGrayLevelEmphasis', 'Adenocarcinoma',
       'Large cell', 'Squamous cell carcinoma', 'NOS', 'age', 'tumor_size'],
      dtype='object')
time: 6.45 s


In [212]:
from lifelines import CoxPHFitter
#Cox
X_train, X_val, Y_train, Y_val = train_test_split(nc_x_train, y_train, test_size=0.15)
df_train = pd.concat([Y_train, X_train], axis = 1)
cph = CoxPHFitter()
cph.fit(df_train, duration_col='SurvivalTime', event_col='Event')

prediction_val = round(cph.predict_expectation(X_val)).astype(int)
print("C_index: " +str(concordance_index(Y_val.SurvivalTime, prediction_val, Y_val.Event)))

C_index: 0.5983772819472617
time: 97.5 ms


/usr/local/lib/python3.6/dist-packages/lifelines/utils/__init__.py:1086: ConvergenceWarning: Column(s) ['original_glrlm_ShortRunLowGrayLevelEmphasis'] have very low variance. This may harm convergence. Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


#Selected Model : DeepSurv

In [215]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel

model = NonLinearCoxPHModel([ {'activation': 'ReLU', 'num_units': 128}, {'activation': 'ReLU', 'num_units': 128}], auto_scaler=True)
#Format Data and fit
E = Y.Event
T = Y.SurvivalTime

model.fit(X, T, E, init_method = 'glorot_uniform', optimizer ='adam',lr = 1e-4, num_epochs = 200, dropout = 0.2, verbose=True, batch_normalization=True)
#Predict
prct = 1/2
hazard = model.predict_survival(x_test)
prediction = np.apply_along_axis(lambda x : model.times[np.argmax(x<prct)], 1, hazard)
prediction[prediction==model.times[0]] = 700
#Write in CSV
PatientID["SurvivalTime"] = prediction.astype(int)
PatientID["Event"] = a = np.full(prediction.shape[0], np.nan)
PatientID.to_csv (r'Prediction_final.csv', index = None, header=True)

time: 1.85 s


In [201]:
PatientID

,PatientID,SurvivalTime,Event
0,13,303,NaN
1,155,700,NaN
2,404,508,NaN
3,407,632,NaN
4,9,700,NaN
...,...,...,...
120,66,493,NaN
121,132,377,NaN
122,169,700,NaN
123,199,575,NaN


time: 26.6 ms
